In [3]:
from first_stage_frlm import first_stage_frlm
from second_stage_frlm import second_stage_frlm
from first_stage_frlm import get_minimal_subsets
import networkx as nx
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import random
import itertools
import pulp

In [4]:
import pickle

G=pickle.load( open('RWS_data/network_digital_twin_v0.2.pickle','rb'))

In [5]:
#Get data from df
df_links = nx.to_pandas_edgelist(G)
df_nodes = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')

In [6]:
df_links.loc[df_links.target == "8866426"]

,source,target,lat_mean,length_m,StartJunctionId,Classification,geometry,PushedWidth,lat_p95,nap_stdDev,...,GeneralWidth,lat_p5,lat_stdDev,nap_p50,PushedDepth,Description,GeneralDepth,nap_count,nap_mean,Name
232,8867817,8866426,NaN,426.400769,8867817,CEMT,"{'type': 'LineString', 'coordinates': ((4.2821...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Vaarwegvak van 0 tot 0 - H


In [7]:
df_links.loc[df_links.Name=='Vaarwegvak van 0 tot 0 - H'].dropna(how='all')
# df_links.loc[df_links.target == "8866426"]
# df_nodes.loc[df_nodes.index=="30534997"].dropna(axais=1)

,source,target,lat_mean,length_m,StartJunctionId,Classification,geometry,PushedWidth,lat_p95,nap_stdDev,...,GeneralWidth,lat_p5,lat_stdDev,nap_p50,PushedDepth,Description,GeneralDepth,nap_count,nap_mean,Name
8,8861581,8863189,NaN,613.104746,8861581,CEMT,"{'type': 'LineString', 'coordinates': ((6.0521...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Vaarwegvak van 0 tot 0 - H
10,8860954,8864050,NaN,1234.583805,8860954,CEMT,"{'type': 'LineString', 'coordinates': ((4.4474...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Zesbaksduwstel,NaN,0,NaN,Vaarwegvak van 0 tot 0 - H
15,8865802,8862540,NaN,576.337751,8865802,CEMT,"{'type': 'LineString', 'coordinates': ((4.4144...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"Groot Rijnschip, Eenbaksduwstel",NaN,0,NaN,Vaarwegvak van 0 tot 0 - H
16,8861605,8863756,NaN,271.985423,8861605,CEMT,"{'type': 'LineString', 'coordinates': ((4.4130...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Vaarwegvak van 0 tot 0 - H
22,8864595,8864137,NaN,116.684543,8864137,CEMT,"{'type': 'LineString', 'coordinates': ((4.6343...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Zesbaksduwstel,NaN,0,NaN,Vaarwegvak van 0 tot 0 - H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14479,Berth20,Berth22,NaN,40.250350,Berth20,CEMT,"{'type': 'LineString', 'coordinates': ((6.0904...",NaN,NaN,0.192602,...,NaN,NaN,NaN,1.380204,NaN,Zesbaksduwstel,NaN,94,1.293976,Vaarwegvak van 0 tot 0 - H
14480,Berth21,8860919,NaN,85.777056,Berth21,CEMT,"{'type': 'LineString', 'coordinates': ((6.0907...",NaN,NaN,0.120221,...,NaN,NaN,NaN,1.541399,NaN,Zesbaksduwstel,NaN,200,1.509271,Vaarwegvak van 0 tot 0 - H
14481,Berth21,Berth22,NaN,5.033140,Berth22,CEMT,"{'type': 'LineString', 'coordinates': ((6.0907...",NaN,NaN,0.025505,...,NaN,NaN,NaN,1.366488,NaN,Zesbaksduwstel,NaN,12,1.375215,Vaarwegvak van 0 tot 0 - H
14487,Berth39,Berth40,NaN,133.630830,Berth39,CEMT,"{'type': 'LineString', 'coordinates': ((3.8259...",NaN,NaN,0.109707,...,16.0,NaN,NaN,-6.017772,NaN,"Groot Rijnschip, Eenbaksduwstel",NaN,302,-6.001109,Vaarwegvak van 0 tot 0 - H


In [8]:
a=df_nodes.index

In [9]:
berths = []
for harbour in a:
    if "Berth" in harbour:
        berths.append(harbour)

## Random origin and destination picked using:

https://deploy-preview-19–digitwin-waterways.netlify.app/#/

(8860852, B43966_B, 10) (8860852, B27067_B, 5)

Now check if it works with two scripts!

In [10]:
df_b, df_g, df_eq_fq = first_stage_frlm(G, 10*(10^3), [("8860852", "8862426", 10)])

{('8860852', '8862426'): 16159.45677737489}


In [11]:
df_b

""
q
"(8860852, 8862426)"


In [12]:
locations = second_stage_frlm(10, 20, df_g, df_b, df_eq_fq)

KeyError: 0